# Comparar tiempos de ejecución para lecturas de archivos geojson #28

Los resultados que se presentan en esta notebook son basados en la misma cantidad de objetos obtenidos a partir de ECATEPEC.json 

# Librería json en Python

Usando la libreria de json en python podemos deserializar los objetos completos en el codigo, pero para este ejemplo solo vamos a importar la propiedad de "coordinates" 
para usarla en los algoritmos actuales.
La funcion cuenta con el timer dentro para medir con exactitud la velocidad con la que deserializa n objetos.

In [4]:
import json
import time

def load_n_coordinates_from_json_file(file_path, n):
    start_time = time.time()  # Start the timer
    
    coordinates_list = []
    with open(file_path, 'r') as file:
        for _ in range(n):
            line = file.readline()
            if not line:
                break
            obj = json.loads(line.strip())
            
            # Extract coordinates
            coordinates = obj["features"][0]["geometry"]["coordinates"]
            coordinates_list.append(coordinates)
    
    end_time = time.time()  # End the timer
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    
    return coordinates_list, elapsed_time

# Resultados

In [ ]:
Para n = 1
Time taken to load 1 coordinates: 0.0001 seconds

Para n = 1000
Time taken to load 1000 coordinates: 0.0522 seconds

Para n = 3000
Time taken to load 3000 coordinates: 0.1773 seconds

Para n = 6000
Time taken to load 6000 coordinates: 0.2746 seconds

# Librería pyspark en Python

Al igual que en la prueba con la liibreria json, la captura del tiempo es dentro de la funcion, la explicacion de la funcionalidad se puede consultar en el notebook de pyspark

In [ ]:
import json
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode

# Inicializa SparkSession
spark = SparkSession.builder.appName("LoadCoordinates").getOrCreate()

def load_n_coordinates_from_json_file(file_path, n):
    start_time = time.time()  # Start the timer
    
    # Lee el archivo JSON usando Spark
    df = spark.read.json(file_path)
    
    # Extrae las coordenadas desde la columna "features"
    coordinates_df = df.select(explode(col("features")).alias("features")).select("features.geometry.coordinates")
    
    # Selecciona las primeras n filas y colecta las coordenadas en una lista
    coordinates_list = coordinates_df.limit(n).rdd.map(lambda row: row[0]).collect()
    
    end_time = time.time()  # End the timer
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    
    return coordinates_list, elapsed_time


# Resultados

In [ ]:
Para n = 1
Time taken to load 1 coordinates: 6.7048 seconds

Para n = 1000
Time taken to load 1000 coordinates: 7.2669 seconds

Para n = 3000
Time taken to load 3000 coordinates: 7.6805 seconds

Para n = 6000
Time taken to load 6000 coordinates: 7.7959 seconds